우선 모델링을 먼저 해보자
model list
1. LinearRegression
2. KNN Regressor
3. SVM
4. Ridge, Lasso
5. 

In [1]:
# 1. Load Data
import pandas as pd

dataDF = pd.read_csv('obesity.csv')
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2064 non-null   int64  
 1   Age                             2064 non-null   int64  
 2   Height                          2064 non-null   float64
 3   Weight                          2064 non-null   float64
 4   family_history_with_overweight  2064 non-null   int64  
 5   FAVC                            2064 non-null   int64  
 6   FCVC                            2064 non-null   int64  
 7   NCP                             2064 non-null   int64  
 8   CAEC                            2064 non-null   int64  
 9   SMOKE                           2064 non-null   int64  
 10  CH2O                            2064 non-null   int64  
 11  SCC                             2064 non-null   int64  
 12  FAF                             20

In [2]:
# 2. Feature Selection
# : ['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC', 'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking', 'NObeyesdad', 'BMI']
# 1) Feature 선별:
#       1. 비만도 제거 NObeyesdad
#       2. 과편향 데이터 제거
dataDF.columns
dataDF.drop(['Weight', 'SMOKE','SCC','MTRANS_Bike', 'MTRANS_Motorbike', 'MTRANS_Walking', 'NObeyesdad'], axis=1, inplace=True)
dataDF.columns

Index(['Gender', 'Age', 'Height', 'family_history_with_overweight', 'FAVC',
       'FCVC', 'NCP', 'CAEC', 'CH2O', 'FAF', 'TUE', 'CALC',
       'MTRANS_Automobile', 'MTRANS_Public_Transportation', 'BMI'],
      dtype='object')

In [3]:
# 2) 컬럼명 수정
dataDF.columns = ['성별', '나이', '키', '가족력', '고열량 섭취 빈도', '채소 섭취 빈도', '하루 섭취 끼니 수', '간식 섭취 빈도', '물 섭취량', '활동량', '스마트폰 사용량', '음주 빈도', '이동 수단 : 자가용', '이동수단 : 대중교통', 'BMI']
dataDF

,성별,나이,키,가족력,고열량 섭취 빈도,채소 섭취 빈도,하루 섭취 끼니 수,간식 섭취 빈도,물 섭취량,활동량,스마트폰 사용량,음주 빈도,이동 수단 : 자가용,이동수단 : 대중교통,BMI
0,0,21,1.620,0,0,2,3,2,2,0,1,3,0,1,24.386526
1,0,21,1.520,0,0,3,3,2,3,3,0,2,0,1,24.238227
2,1,23,1.800,0,0,2,3,2,2,2,1,1,0,1,23.765432
3,1,27,1.800,1,0,3,3,2,2,2,0,1,0,0,26.851852
4,1,22,1.780,1,0,2,1,2,2,0,0,2,0,1,28.342381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,0,21,1.711,0,1,3,3,2,2,2,1,2,0,1,44.884392
2060,0,22,1.749,0,1,3,3,2,2,1,1,2,0,1,43.707080
2061,0,23,1.752,0,1,3,3,2,2,1,1,2,0,1,43.557526
2062,0,24,1.739,0,1,3,3,2,3,1,1,2,0,1,44.078924


In [4]:
# 3) 인사이트 : 상관계수, 분포도

In [5]:
# 4) Feature, Target Split
feature = dataDF.drop('BMI', axis=1)
target = dataDF['BMI']
print(feature.shape, target.shape)

# 5) train, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=11)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2064, 14) (2064,)


(1651, 14) (413, 14) (1651,) (413,)


In [6]:
# 6) Standardize : '나이'만 정규화하기
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled, X_test_scaled = X_train.copy(), X_test.copy()
scaler.fit(X_train_scaled[['나이']])
X_train_scaled['나이'] = scaler.transform(X_train_scaled['나이'].values.reshape(-1,1))
X_test_scaled['나이'] = scaler.transform(X_test_scaled['나이'].values.reshape(-1,1))
X_test_scaled

c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,성별,나이,키,가족력,고열량 섭취 빈도,채소 섭취 빈도,하루 섭취 끼니 수,간식 섭취 빈도,물 섭취량,활동량,스마트폰 사용량,음주 빈도,이동 수단 : 자가용,이동수단 : 대중교통
785,0,-0.980622,1.737,0,1,3,3,2,2,1,1,1,0,1
151,1,1.490673,1.690,0,0,2,3,2,3,2,0,3,1,0
164,0,0.718393,1.740,0,0,3,3,0,2,0,0,2,1,0
279,1,-0.517254,1.790,0,1,2,3,0,1,0,0,2,0,1
188,1,-0.671710,1.780,1,0,2,1,1,3,2,1,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1,0.718393,1.766,0,1,1,4,2,2,1,1,2,0,1
1737,1,0.100569,1.762,0,1,1,3,2,2,1,0,2,0,1
53,0,-0.208342,1.630,0,0,3,3,3,2,2,1,3,0,1
220,0,-0.671710,1.600,1,1,2,3,2,1,0,1,3,1,0


In [7]:
# 3. Model Selection
# 1) KNN Regression
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor()
knn_model.fit(X_train_scaled, y_train)

# 2) Linear Regression
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)

# 3) SVM
from sklearn.svm import SVR

svm_model = SVR()
svm_model.fit(X_train_scaled, y_train)

# 4. Evaluation
print(f'[knn] train : {knn_model.score(X_train_scaled, y_train)}, test : {knn_model.score(X_test_scaled, y_test)}')
print(f'[linear] train : {linear_model.score(X_train_scaled, y_train)}, test : {linear_model.score(X_test_scaled, y_test)}')
print(f'[svm] train : {svm_model.score(X_train_scaled, y_train)}, test : {svm_model.score(X_test_scaled, y_test)}')

[knn] train : 0.8469570621816936, test : 0.7421718468857268
[linear] train : 0.45898049219526815, test : 0.4000346467827014
[svm] train : 0.5788774349736362, test : 0.5282280687985028


knn, svm 은 생각보다 잘 나온다!  
정규화 수정하니 svm도 떡락  
linear 는 대부분 범주화 데이터라 점수가 낮은 듯

In [8]:
# 5. Tunning
# 1) knn
from sklearn.model_selection import GridSearchCV

knn_params = [
    {
        'n_neighbors' : [3, 5, 7, 9, 11],
        'weights' : ['uniform', 'distance'],
        'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
]

knn_model = KNeighborsRegressor()
knn_cv = GridSearchCV(knn_model, knn_params, cv=5)
knn_cv.fit(X_train_scaled, y_train)
print(f'[knn] best_score : {knn_cv.best_score_}, best_params : {knn_cv.best_params_}')


[knn] best_score : 0.7582882197125387, best_params : {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'distance'}


In [9]:
# 2) svm
from sklearn.model_selection import GridSearchCV

svm_params = [
    {
        'C' : [1, 10, 100, 1000],
        'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
        'gamma' : ['scale', 'auto']
    }
]
svm_model = SVR()
svm_cv = GridSearchCV(svm_model, svm_params, cv=5)
svm_cv.fit(X_train_scaled, y_train)
print(f'[svm] best_score : {svm_cv.best_score_}, best_params : {svm_cv.best_params_}')

print(svm_model)

: 

6분이나 걸림,, 0.814

In [1]:
# all estimators
from sklearn.utils import all_estimators

allAlgorithms = all_estimators(type_filter='regressor')
# print(allAlgorithms)
all_regressor = []
all_regressor_name = []
# c = 0
for name, model in allAlgorithms:
    print(f'[{name}] : {model}')
    all_regressor.append(model)
    all_regressor_name.append(name)
    # c += 1
# print(c)

[ARDRegression] : <class 'sklearn.linear_model._bayes.ARDRegression'>
[AdaBoostRegressor] : <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>
[BaggingRegressor] : <class 'sklearn.ensemble._bagging.BaggingRegressor'>
[BayesianRidge] : <class 'sklearn.linear_model._bayes.BayesianRidge'>
[CCA] : <class 'sklearn.cross_decomposition._pls.CCA'>
[DecisionTreeRegressor] : <class 'sklearn.tree._classes.DecisionTreeRegressor'>
[DummyRegressor] : <class 'sklearn.dummy.DummyRegressor'>
[ElasticNet] : <class 'sklearn.linear_model._coordinate_descent.ElasticNet'>
[ElasticNetCV] : <class 'sklearn.linear_model._coordinate_descent.ElasticNetCV'>
[ExtraTreeRegressor] : <class 'sklearn.tree._classes.ExtraTreeRegressor'>
[ExtraTreesRegressor] : <class 'sklearn.ensemble._forest.ExtraTreesRegressor'>
[GammaRegressor] : <class 'sklearn.linear_model._glm.glm.GammaRegressor'>
[GaussianProcessRegressor] : <class 'sklearn.gaussian_process._gpr.GaussianProcessRegressor'>
[GradientBoostingRegressor] : 

In [ ]:
# regression 재정의
all_regressor_name =[ 'AdaBoostRegressor',
                    'DecisionTreeRegressor',
                    'ElasticNetCV',
                    'ExtraTreeRegressor',
                    'ExtraTreesRegressor',
                    'GradientBoostingRegressor',
                    'KNeighborsRegressor',
                    'KernelRidge',
                    'Lasso',
                    'LassoCV',
                    'LinearRegression',
                    'LinearSVR',
                    'RandomForestRegressor',
                    'Ridge',
                    'RidgeCV',
                    'SGDRegressor',
                    'SVR',
                    'VotingRegressor']

all_regressor = []
for name, model in allAlgorithms:
    if name in all_regressor_name:
        all_regressor.append((name, model))

In [ ]:
# all regression evaluate
from sklearn.model_selection import cross_val_score

score_list = []
for name, model in all_regressor:
    try:
        print(f'[{name}]')
        md = model()
        md.fit(X_train_scaled, y_train)
        print('Score ', md.score(X_test_scaled, y_test))
        score_list.append((name, md.score(X_test_scaled, y_test)))
    except Exception as e:
        print(e)
print('End')

# Score 높은 순으로 세 모델
score_list.sort(key=lambda x: x[1], reverse=True)
print(score_list[:3])

best_score = max(score_list, key=lambda x: x[1])
print(f'Best Score : {best_score}')

[AdaBoostRegressor]
Score  0.5885992251606145
[DecisionTreeRegressor]
Score  0.7320494063138357
[ElasticNetCV]
Score  0.39994725117277485
[ExtraTreeRegressor]
Score  0.6838247693132053
[ExtraTreesRegressor]
Score  0.8775413633939323
[GradientBoostingRegressor]
Score  0.7015132315628385
[KNeighborsRegressor]
Score  0.7421718468857268
[KernelRidge]
Score  0.3998938153156105
[Lasso]
Score  0.15734996441653848
[LassoCV]
Score  0.4001818177887383
[LinearRegression]
Score  0.4000346467827014
[LinearSVR]
Score  0.36594793309489426
[RandomForestRegressor]


c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Score  0.8633857090306292
[Ridge]
Score  0.4000565544704515
[RidgeCV]
Score  0.40005655447041744
[SGDRegressor]
Score  0.3901249405632773
[SVR]
Score  0.5282280687985028
[VotingRegressor]
__init__() missing 1 required positional argument: 'estimators'
End
[('ExtraTreesRegressor', 0.8775413633939323), ('RandomForestRegressor', 0.8633857090306292), ('KNeighborsRegressor', 0.7421718468857268)]
Best Score : ('ExtraTreesRegressor', 0.8775413633939323)


Top3  
[('ExtraTreesRegressor', 0.8775413633939323), ('RandomForestRegressor', 0.8633857090306292), ('KNeighborsRegressor', 0.7421718468857268)]

In [ ]:
# ExtraTreesRegressor 점수 확인
from sklearn.ensemble import ExtraTreesRegressor
et_reg = ExtraTreesRegressor()
et_reg.fit(X_train_scaled, y_train)
print(f'ExtraTreesRegressor train score : {et_reg.score(X_train_scaled, y_train)}')
print(f'ExtraTreesRegressor test score : {et_reg.score(X_test_scaled, y_test)}')

ExtraTreesRegressor train score : 0.9998993752730202
ExtraTreesRegressor test score : 0.8804351120902443


ExtraTreesRegressor 에서 가장 높은 성능을 보여준다.

In [ ]:
# GridSearchCV 활용
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [30, 100, 150], # 생성 트리 수
     'max_depth': [None, 10, 20],     # 트리의 최대 깊이
     'min_samples_split': [2, 10],      # 내부 노드 분할에 필요한 최소 샘플 수
     'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson']   # 분할 품질 측정 기능 기준
    }
]

gcv = GridSearchCV(estimator=ExtraTreesRegressor(), param_grid=param_grid, cv=5)
gcv.fit(X_train_scaled, y_train)
print(gcv.best_params_)
print(gcv.best_estimator_)
print(gcv.best_score_)

: 

교차 검증으로 평균을 내니 점수가 더 떨어진 것으로 예상

In [ ]:
# RandomForest
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(X_train_scaled, y_train)
print(f'RandomForestRegressor train score : {rfr.score(X_train_scaled, y_train)}')
print(f'RandomForestRegressor test score : {rfr.score(X_test_scaled, y_test)}')


RandomForestRegressor train score : 0.9798564535408443
RandomForestRegressor test score : 0.862836140882383


Predict

Save Model


In [ ]:
#